In [137]:
import numpy as np
import seaborn as sns
import pandas as pd
import requests
 from bs4 import BeautifulSoup
from selenium import webdriver
import time

# importing dataframes

In [138]:
season1819 = pd.read_csv('season1920.csv')

In [139]:
season1819 = season1819.drop_duplicates(subset = 'Player', keep = 'first')
season1819 = season1819.drop(['Unnamed: 0', 'Unnamed: 29'], axis = 1)

In [140]:
season1819['url_list'][1]

'https://www.basketball-reference.com/players/a/acyqu01.html'

# scraping from front page 

In [71]:
url = 'https://www.basketball-reference.com/'
res = requests.get(url)
res.status_code

200

In [98]:
teams = []
def sports_page(url):
    res = requests.get(url)
    soup = BeautifulSoup(res.content, 'lxml')
    wc_table = soup.find('table', {'id' : 'confs_standings_W'})
    ec_table = soup.find('table', {'id' : 'confs_standings_E'})
    
    #wc teams
    for row in wc_table.find_all('tr')[1:]:
        wc_team = {}
        wc_team['slug'] = row.find('a').attrs['title']
        wc_team['link'] = row.find('a').attrs['href']
        teams.append(wc_team)
    #ec teams
    for row in ec_table.find_all('tr')[1:]:
        ec_team = {}
        ec_team['slug'] = row.find('a').attrs['title']
        ec_team['link'] = row.find('a').attrs['href']
        teams.append(ec_team)
        
    df = pd.DataFrame(teams)
        
        
    return df
        
team_links = 'https://www.basketball-reference.com/'

In [101]:
df.to_csv('data/team_links.csv')

## find salary table on team page

This will be salary based on the 2019-2020 season

In [134]:
url = 'https://www.basketball-reference.com'
driver = webdriver.Chrome(executable_path="./chromedriver/windows/chromedriver.exe")
play = []
for page in df['link']:
    time.sleep(3)
    team_page = url + page
    driver.get(team_page)
    html = driver.page_source
    html = BeautifulSoup(html, 'lxml')
    ad_table = html.find('table', {'id' : 'salaries2'})
    chart = ad_table.find('tbody')
    players = chart.find_all('tr')
    for stat in players:
        number = [row.text for row in stat.find_all('td')]
        play.append(number)

In [149]:
salaries = pd.DataFrame(play)
salaries.columns = ['Player', 'salary']
salaries.to_csv('data/salaries.csv')

,Player,salary
0,LeBron James,"$37,436,858"
1,Anthony Davis,"$27,093,019"
2,Danny Green,"$14,634,146"
3,Kentavious Caldwell-Pope,"$8,089,282"
4,Luol Deng,"$4,990,000"


# Matching seaon1819 dataframe to salaries

In [144]:
print(len(season1819))
print(len(salaries))

531
491


In [150]:
merged_df = season1819.merge(salaries, how = 'outer', on='Player')

In [152]:
merged_df.dropna(inplace = True)

In [153]:
merged_df

,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,...,AST,STL,BLK,TOV,PF,PTS,ORtg,DRtg,url_list,salary
2,Jaylen Adams,PG,22.0,ATL,34.0,1.0,428.0,4.1,11.9,0.345,...,7.0,1.5,0.5,3.0,4.9,11.7,99.0,115.0,https://www.basketball-reference.com/players/a...,"$100,000"
3,Steven Adams,C,25.0,OKC,80.0,80.0,2669.0,8.4,14.1,0.595,...,2.2,2.0,1.3,2.4,3.6,19.4,120.0,106.0,https://www.basketball-reference.com/players/a...,"$25,842,697"
4,Bam Adebayo,C,21.0,MIA,82.0,28.0,1913.0,7.2,12.4,0.576,...,4.7,1.8,1.7,3.1,5.2,18.6,120.0,104.0,https://www.basketball-reference.com/players/a...,"$3,454,080"
7,LaMarcus Aldridge,C,33.0,SAS,81.0,81.0,2687.0,12.4,24.0,0.519,...,3.5,0.8,1.9,2.6,3.3,31.4,117.0,110.0,https://www.basketball-reference.com/players/a...,"$26,000,000"
9,Grayson Allen,SG,23.0,UTA,38.0,2.0,416.0,7.7,20.5,0.376,...,2.9,0.7,0.7,3.8,5.4,24.3,95.0,111.0,https://www.basketball-reference.com/players/a...,"$2,429,400"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
529,Delon Wright,PG,26.0,TOT,75.0,13.0,1699.0,6.9,16.0,0.434,...,7.1,2.5,0.9,2.2,3.0,18.7,112.0,107.0,https://www.basketball-reference.com/players/w...,"$9,473,684"
530,Guerschon Yabusele,PF,23.0,BOS,41.0,1.0,251.0,6.7,14.8,0.455,...,2.9,1.5,1.3,2.9,6.1,18.0,108.0,109.0,https://www.basketball-reference.com/players/y...,"$3,117,240"
532,Thaddeus Young,PF,30.0,IND,81.0,81.0,2489.0,8.7,16.5,0.527,...,4.0,2.4,0.7,2.4,3.8,20.1,113.0,106.0,https://www.basketball-reference.com/players/y...,"$12,900,000"
533,Trae Young,PG,20.0,ATL,81.0,81.0,2503.0,9.7,23.2,0.418,...,12.1,1.3,0.3,5.7,2.6,28.6,107.0,117.0,https://www.basketball-reference.com/players/y...,"$6,273,000"
